In [ ]:
# import packages 
import pandas as pd
import numpy as np
import pandasql as psql
import re
import os

In [ ]:
# datasets
# maternal mortality and health indicators by county 
maternal_mortality = pd.read_csv('data/processed/maternal_mortality_country_all_race_all_age_2019.csv')
health_indicators = pd.read_csv('data/processed/merged_health_indicators.csv')

# state level political data 
abortion_laws_index = pd.read_csv('data/raw/abortion_laws_index.csv')
state_gov_control = pd.read_csv('data/processed/state_gov_control_2017.csv')

# states data to help join state_abbrv<->state name 
states = pd.read_csv('data/raw/project_political_data - States.csv')


In [ ]:
# check to make sure we're not missing any states
maternal_mortality.state_name.nunique()

50

In [ ]:
#limit health indicators data to target control variables

health_indicators_target_var = health_indicators[['county_fips','low_income_pct','obgyn_provider_rate_100k',
                                                  'prenatal_care_first_trimester_pct','obesity_pre_pregnancy_pct',
                                                  'diabetes_pre_pregnancy_pct', 'hypertension_pre_pregnancy_pct' ]]

In [ ]:
# merge health indicators with maternal mortality on county_fips
maternal_mortality = pd.merge(maternal_mortality,health_indicators_target_var, how = 'left', left_on ='fips', right_on = 'county_fips')

In [64]:
# merge states data to help with state abbr <-> state name joins 
maternal_mortality = pd.merge(maternal_mortality, states, how ='left', left_on = 'state_name', right_on ='State')

In [ ]:
#limit abortion laws data to target variables
abortion_laws_index_target_var = abortion_laws_index[['State Abbreviation','guttmacher_index']]

# merge abortion laws with maternal mortality on state abbreviation
maternal_mortality = pd.merge(maternal_mortality, abortion_laws_index_target_var, how ='left', left_on = 'State Abbreviation', right_on ='State Abbreviation')

In [ ]:
# limit state gov control to target variables
state_gov_control_target_var = state_gov_control[['state','gop_trifecta']]

# merge abortion laws with maternal mortality on state name
maternal_mortality = pd.merge(maternal_mortality, state_gov_control_target_var, how ='left', left_on = 'State', right_on ='state')

In [ ]:
# review resulting DF
maternal_mortality.head()

,Unnamed: 0,measure_id,measure_name,location_id,location_name,fips,race_id,race_name,sex_id,sex_name,...,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,State Abbreviation,State,guttmacher_index,state,gop_trifecta
0,3648030,1,Deaths,614,Autauga County (Alabama),1001.0,1,Total,2,Female,...,3.5,71.8,32.1,1.1,3.0,AL,Alabama,most_restrictive,Alabama,1
1,3648036,1,Deaths,637,Baldwin County (Alabama),1003.0,1,Total,2,Female,...,19.4,78.3,28.0,1.2,3.3,AL,Alabama,most_restrictive,Alabama,1
2,3648042,1,Deaths,624,Barbour County (Alabama),1005.0,1,Total,2,Female,...,0.0,67.4,41.2,1.1,4.6,AL,Alabama,most_restrictive,Alabama,1
3,3648048,1,Deaths,603,Bibb County (Alabama),1007.0,1,Total,2,Female,...,0.0,66.4,37.5,1.1,3.1,AL,Alabama,most_restrictive,Alabama,1
4,3648054,1,Deaths,588,Blount County (Alabama),1009.0,1,Total,2,Female,...,3.4,72.3,32.8,1.2,2.9,AL,Alabama,most_restrictive,Alabama,1


In [ ]:
# Check for null values
maternal_mortality.isna().sum()

Unnamed: 0                             0
measure_id                             0
measure_name                           0
location_id                            0
location_name                          0
fips                                   0
race_id                                0
race_name                              0
sex_id                                 0
sex_name                               0
age_group_id                           0
age_name                               0
cause_id                               0
cause_name                             0
year                                   0
metric_id                              0
metric_name                            0
val                                   31
upper                                 31
lower                                 31
state_name                             0
county_fips                           10
low_income_pct                        19
obgyn_provider_rate_100k              13
prenatal_care_fi

In [ ]:
# More data checks on resulting DF 
maternal_mortality.describe()

,Unnamed: 0,measure_id,location_id,fips,race_id,sex_id,age_group_id,cause_id,year,metric_id,...,upper,lower,county_fips,low_income_pct,obgyn_provider_rate_100k,prenatal_care_first_trimester_pct,obesity_pre_pregnancy_pct,diabetes_pre_pregnancy_pct,hypertension_pre_pregnancy_pct,gop_trifecta
count,3.112000e+03,3112.0,3112.000000,3112.000000,3112.0,3112.0,3112.0,3112.0,3112.0,3112.0,...,3081.000000,3081.000000,3102.000000,3093.000000,3099.000000,3099.000000,3099.000000,3008.000000,3069.000000,3112.000000
mean,3.845659e+06,1.0,3964.326157,30053.951157,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000049,0.000025,30112.270471,26.905690,9.809100,76.683349,32.025653,1.056217,2.351548,0.664846
std,1.164816e+05,0.0,12880.776428,15129.727079,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000018,0.000010,15087.083227,9.466455,13.519115,6.934325,4.891153,0.394139,1.097705,0.472120
min,3.648030e+06,1.0,574.000000,1001.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000010,0.000004,1001.000000,0.000000,0.000000,36.700000,8.700000,0.200000,0.100000,0.000000
25%,3.742156e+06,1.0,1378.750000,18144.500000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000037,0.000018,18155.500000,20.100000,0.000000,72.500000,29.400000,0.800000,1.500000,0.000000
50%,3.845157e+06,1.0,2160.500000,29130.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000046,0.000023,29136.000000,25.800000,4.000000,77.800000,32.400000,1.000000,2.200000,1.000000
75%,3.943946e+06,1.0,2942.250000,45023.500000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000057,0.000030,45024.500000,32.600000,16.400000,81.600000,35.000000,1.200000,2.900000,1.000000
max,4.044456e+06,1.0,94092.000000,56045.000000,1.0,2.0,22.0,962.0,2019.0,3.0,...,0.000182,0.000088,56045.000000,73.300000,172.500000,93.000000,54.700000,10.700000,8.500000,1.000000


In [ ]:
# export main DF to csv
maternal_mortality.to_csv('data/processed/maternal_mortality_complete_dataset.csv')